## Cleaning not-null values

In [1]:
import numpy as np
import pandas as pd

In [3]:
df = pd.DataFrame({
    'Sex': ['M', 'F', 'F', 'D', '?'],
    'Age': [29, 30, 24, 290, 25],
})
df

,Sex,Age
0,M,29
1,F,30
2,F,24
3,D,290
4,?,25



O `DataFrame` anterior não tem nenhum "valor ausente", mas claramente possui dados inválidos. 290 não parece ser uma idade válida, e D e ? não correspondem a nenhuma categoria de sexo conhecida. Como você pode limpar esses valores não ausentes, mas claramente inválidos?

### Finding Unique Values

O primeiro passo para limpar valores inválidos é **percebê-los**, depois **identificá-los** e, finalmente, lidar com eles adequadamente (removê-los, substituí-los, etc). Geralmente, para um campo do tipo "categórico" (como Sexo, que só aceita valores de um conjunto discreto `('M', 'F')`), começamos analisando a variedade de valores presentes. Para isso, usamos o método `unique()`:

In [4]:
df['Sex'].unique()

array(['M', 'F', 'D', '?'], dtype=object)

In [5]:
df['Sex'].value_counts()

Sex
F    2
M    1
D    1
?    1
Name: count, dtype: int64

Claramente, se você ver valores como 'D' ou '?', isso imediatamente chamará sua atenção. Agora, o que fazer com eles? Você pode usar a função `replace` para substituir esses valores:

In [6]:
df['Sex'].replace('D', 'F')

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

Também aceita um dicionário de valores para substituir:

In [7]:
df['Sex'].replace({'D': 'F', 'N': 'M'})

0    M
1    F
2    F
3    F
4    ?
Name: Sex, dtype: object

Se você tiver muitas colunas para substituir, você pode aplicá-lo no "DataFrame level":

In [8]:
df.replace({
    'Sex': {
        'D': 'F',
        'N': 'M'
    },
    'Age': {
        290: 29
    }
})

,Sex,Age
0,M,29
1,F,30
2,F,24
3,F,29
4,?,25


### Duplicates

Verificar valores duplicados é extremamente simples. Isso se comportará de forma diferente entre Series e DataFrames. Vamos começar com Series. Como exemplo, digamos que estamos organizando uma festa chique e estamos convidando embaixadores da Europa. Mas só podemos convidar um embaixador por país. Esta é nossa lista original e, como você pode ver, tanto o Reino Unido quanto a Alemanha têm embaixadores duplicados:

In [9]:
ambassadors = pd.Series([
    'France',
    'United Kingdom',
    'United Kingdom',
    'Italy',
    'Germany',
    'Germany',
    'Germany',
], index=[
    'Gérard Araud',
    'Kim Darroch',
    'Peter Westmacott',
    'Armando Varricchio',
    'Peter Wittig',
    'Peter Ammon',
    'Klaus Scharioth '
])

ambassadors

Gérard Araud                  France
Kim Darroch           United Kingdom
Peter Westmacott      United Kingdom
Armando Varricchio             Italy
Peter Wittig                 Germany
Peter Ammon                  Germany
Klaus Scharioth              Germany
dtype: object

Os dois métodos mais importantes para lidar com duplicatas são `duplicated` (que informará quais valores são duplicatas) e `drop_duplicates` (que simplesmente se livrará das duplicatas):

In [10]:
ambassadors.duplicated()

Gérard Araud          False
Kim Darroch           False
Peter Westmacott       True
Armando Varricchio    False
Peter Wittig          False
Peter Ammon            True
Klaus Scharioth        True
dtype: bool

Neste caso, o método `duplicated` não considerou 'Kim Darroch', a primeira instância do Reino Unido, ou 'Peter Wittig' como duplicatas. Isso ocorre porque, por padrão, ele considerará a primeira ocorrência do valor como não duplicata. Você pode alterar esse comportamento com o parâmetro `keep`:

In [11]:
ambassadors.duplicated(keep='last')

Gérard Araud          False
Kim Darroch            True
Peter Westmacott      False
Armando Varricchio    False
Peter Wittig           True
Peter Ammon            True
Klaus Scharioth       False
dtype: bool

Neste caso, o resultado é "invertido", 'Kim Darroch' e 'Peter Wittig' (os primeiros embaixadores de seus países) são considerados duplicatas, mas 'Peter Westmacott' e 'Klaus Scharioth' não são duplicatas. Você também pode optar por marcar todos eles como duplicatas com `keep=False`:

In [ ]:
ambassadors.duplicated(keep=False)


Um método semelhante é `drop_duplicates`, que simplesmente exclui os valores duplicados e também aceita o parâmetro keep:

In [ ]:
ambassadors.drop_duplicates()
ambassadors.drop_duplicates(keep='last')
ambassadors.drop_duplicates(keep=False)

### Duplicates in DataFrames

Conceptualmente falando, duplicatas em um DataFrame ocorrem no nível de "linha". Duas linhas com exatamente os mesmos valores são consideradas duplicatas:

In [12]:
players = pd.DataFrame({
    'Name': [
        'Kobe Bryant',
        'LeBron James',
        'Kobe Bryant',
        'Carmelo Anthony',
        'Kobe Bryant',
    ],
    'Pos': [
        'SG',
        'SF',
        'SG',
        'SF',
        'SF'
    ]
})

players

,Name,Pos
0,Kobe Bryant,SG
1,LeBron James,SF
2,Kobe Bryant,SG
3,Carmelo Anthony,SF
4,Kobe Bryant,SF



No DataFrame anterior, fica claro que Kobe está duplicado; mas ele aparece com duas posições diferentes. O que `duplicated` diz?

In [13]:
players.duplicated()

0    False
1    False
2     True
3    False
4    False
dtype: bool


Novamente, conceitualmente, "duplicado" significa "todos os valores das colunas devem ser duplicados". Podemos personalizar isso com o parâmetro `subset`:

In [14]:
players.duplicated(subset=['Name'])

0    False
1    False
2     True
3    False
4     True
dtype: bool

E as mesmas regras de `keep` ainda se aplicam:

In [15]:
players.duplicated(subset=['Name'], keep='last')

0     True
1    False
2     True
3    False
4    False
dtype: bool

`drop_duplicates` recebe os mesmos parâmetros:

In [16]:
players.drop_duplicates()
players.drop_duplicates(subset=['Name'])
players.drop_duplicates(subset=['Name'], keep='last')

,Name,Pos
1,LeBron James,SF
3,Carmelo Anthony,SF
4,Kobe Bryant,SF
